### Bi-direction RNN

In [1]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data


In [3]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

n_inputs = 28

n_steps = 28

n_hidden = 128
n_classes = 10

W0928 12:20:46.382730  6948 deprecation.py:323] From <ipython-input-3-a4a535036686>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0928 12:20:46.385722  6948 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0928 12:20:46.386719  6948 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Pl

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz


W0928 12:20:46.594017  6948 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [5]:
x = tf.placeholder('float', [None, n_steps, n_inputs])
y = tf.placeholder('float',[None, n_classes])
weights = {
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes])) # 이중RNN이라
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [11]:
# 함수의  reuturn 없는 이유? 데이터 타입이 없기 때문
def BiRNN(x, weights, biases):
    # 마지막 것 사용을 위해
    x = tf.transpose(x, [1,0,2])
    
    # 1차원 생성: flatten 하는 것
    x = tf.reshape(x, [-1,n_inputs])
    # 28
    x = tf.split(axis=0, num_or_size_splits = n_steps, value=x)
    
    # lstm gate종류: forget-gate, input-gate, output-gate
    # weights가 있음 반드시  biases 있다
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    try: # _ return 형식값 맞춰주기 위해 _ 씀 (fw_state, bw_state)
        outputs,_,_ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
        
    except Exception:
        # 들어간 이미지 개수만큼  outputs
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    
    # output 결과 중 마지막 하나 사용
    # 앞뒤로 생성되는 값 256개 출력 * (256x10) => 10개 특성으로 빠짐
    return tf.matmul(outputs[-1], weights['out'] + biases['out']) #weight 128x2 = 256 x 10 가중치

In [12]:
pred = BiRNN(x, weights, biases)

W0928 12:37:54.962653  6948 deprecation.py:323] From <ipython-input-11-9d4b4da6dc9b>:17: static_bidirectional_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
W0928 12:37:54.963621  6948 deprecation.py:323] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn.py:1610: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
W0928 12:37:54.979577  6948 deprecation.py:506] From C:\Users\KITCOOP\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer i

In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

W0928 12:39:23.584550  6948 deprecation.py:323] From <ipython-input-13-7db698dcc7d3>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [17]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    step=1
    while step*batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_inputs))
        
        sess.run(optimizer, feed_dict = {x:batch_x, y:batch_y})
        if step%display_step == 0:
            acc =sess.run(accuracy, feed_dict={x:batch_x, y:batch_y})
            loss = sess.run(cost, feed_dict={x:batch_x, y:batch_y})
            print('Iter'+str(step*batch_size)+'\nMinibatch Loss='
                 + '{:.6f}'.format(loss)+', Training Accuracy='
                 + '{:.5f}'.format(acc))
        step += 1
    print('Optimization Finished')
    
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1,n_steps, n_inputs))
    test_label = mnist.test.labels[:test_len]
    print('Testing Accuracy:', sess.run(accuracy, feed_dict={x:test_data, y:test_label}))

Iter1280
Minibatch Loss=1.892155, Training Accuracy=0.37500
Iter2560
Minibatch Loss=1.503160, Training Accuracy=0.39844
Iter3840
Minibatch Loss=1.436678, Training Accuracy=0.49219
Iter5120
Minibatch Loss=0.994916, Training Accuracy=0.65625
Iter6400
Minibatch Loss=0.754417, Training Accuracy=0.78906
Iter7680
Minibatch Loss=0.775053, Training Accuracy=0.78125
Iter8960
Minibatch Loss=0.583514, Training Accuracy=0.80469
Iter10240
Minibatch Loss=0.519098, Training Accuracy=0.82031
Iter11520
Minibatch Loss=0.559341, Training Accuracy=0.82031
Iter12800
Minibatch Loss=0.511626, Training Accuracy=0.79688
Iter14080
Minibatch Loss=0.336857, Training Accuracy=0.89844
Iter15360
Minibatch Loss=0.435611, Training Accuracy=0.88281
Iter16640
Minibatch Loss=0.423179, Training Accuracy=0.89062
Iter17920
Minibatch Loss=0.327444, Training Accuracy=0.90625
Iter19200
Minibatch Loss=0.269810, Training Accuracy=0.91406
Iter20480
Minibatch Loss=0.282459, Training Accuracy=0.87500
Iter21760
Minibatch Loss=0.2499